# Análise de Dados Aprendizado de Máquina - NBA

_descrever objetivo do trabalho e etc_

## Informações do Aluno:

* Nome: Gabriel Castelo Branco Rocha Alencar Pinto
* Matrícula: 2020006523


## Aprendizado supervisionado



In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import modules.supervisioned as sup
import importlib

plt.style.use("seaborn")

plt.rcParams['figure.figsize']  = (16, 10)
plt.rcParams['axes.labelsize']  = 20
plt.rcParams['axes.titlesize']  = 20
plt.rcParams['legend.fontsize'] = 20
plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20
plt.rcParams['lines.linewidth'] = 4


def despine(ax=None):
    if ax is None:
        ax = plt.gca()
        
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    
%load_ext autoreload
%autoreload 2

/tmp/ipykernel_14246/936430673.py:6: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn")


In [2]:
df_treino = pd.read_csv("./data/nba_treino.csv")
df_treino.head()

df_treino['TARGET_5Yrs'] = df_treino["TARGET_5Yrs"].astype(bool)
# df_treino.value_counts()

In [3]:
df_teste = pd.read_csv("./data/nba_teste.csv")
df_teste.head()

df_teste['TARGET_5Yrs'] = df_teste["TARGET_5Yrs"].astype(bool)

In [4]:
classifier = sup.KNNClassifier(df_treino, "TARGET_5Yrs", 5)

classifier.train()

In [10]:
# classifier.print_point_info(1071)
classifier.predict_point(df_teste.iloc[200])

# pd.DataFrame(df_teste.iloc[0]).drop('TARGET_5Yrs')

[(0, 38.88650151402154, False), (1, 41.264997273718556, False), (2, 9.158602513484254, False), (3, 18.116291011131395, True), (4, 42.760028063601645, True)]
KNN do ponto 1072
Vizinho 0: 38.88650151402154
Vizinho 1: 41.264997273718556
Vizinho 2: 9.158602513484254
Vizinho 3: 18.116291011131395
Vizinho 4: 42.760028063601645
Classificação: False. Real: False
